# Worksheet 05

Name:  Zhupei Xu
UID: U83632486

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

cluster:[0, .5] [1.5, 2, 6, 6.5, 7]

new centroids: [0.25, 4.6]
cluster: [0, .5, 1.5, 2] [6, 6.5, 7]

new centroids: [1, 6.5]
cluster: [0, .5, 1.5, 2] [6, 6.5, 7]


b) Describe in plain english what the cost function for k means is.

The cost function of K-means represents the total distance of all points to the center of their assigned group.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Random initialization for the inital centroids.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes. There are only so many partions available. It will eventually reach to a point and stay unchanged. It cannot go back to a previously defined centroids.

e) Follow along in class the implementation of Kmeans

In [3]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

   


    def lloyds(self):
        indices = np.random.choice(len(self.data), self.k, replace=False)
        centers = self.data[indices]

        while True:
            for i, point in enumerate(self.data):
                distances = np.sum((centers - point) ** 2, axis=1)
                self.assignment[i] = np.argmin(distances)

            self.snap(centers)
            
            new_centers = []
            for j in range(self.k):
                assigned_points = self.data[np.array(self.assignment) == j]
                if len(assigned_points) > 0: 
                    new_center = np.mean(assigned_points, axis=0)
                    new_centers.append(new_center)
                else: 
                    new_centers.append(self.data[np.random.choice(len(self.data))])
            
            new_centers = np.array(new_centers)

          
            if np.allclose(centers, new_centers):
                break
            
            centers = new_centers
        return

            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)